In [1]:
# ---------------------------------------------------------
# SCRIPT: sandbox.py
# DESCRIPCIÓN: Estudio de los datos y su estructura
# ---------------------------------------------------------

from pyspark.sql import SparkSession

# Configuramos la sesión (el "driver") para que sepa hablar con MinIO
print("🔌 Conectando Spark con el Data Lake...")

spark = SparkSession.builder \
    .appName("SkyTracker_Exploracion") \
    .master("local[*]") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()

print("✅ Spark está listo para recibir órdenes.")

🔌 Conectando Spark con el Data Lake...
✅ Spark está listo para recibir órdenes.


In [2]:
# Leemos el archivo tal cual es (Raw)
print("📂 Leyendo 'flights.csv' desde la capa Bronze...")

df_raw = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("s3a://bronze/raw_data/flights.csv")

# Imprimimos la estructura 
print("\n--- 📋 ESQUEMA DE DATOS (COLUMNAS Y TIPOS) ---")
df_raw.printSchema()

# Vemos un ejemplo real (las primeras 2 filas) para entender qué hay dentro
print("\n--- 👀 VISTA PREVIA DE LOS DATOS ---")
df_raw.show(2, vertical=True) 

📂 Leyendo 'flights.csv' desde la capa Bronze...

--- 📋 ESQUEMA DE DATOS (COLUMNAS Y TIPOS) ---
root
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: integer (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: integer (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- WHEELS_OFF: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- WHEELS_ON: integer (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (

In [ ]:
# --- SELECCIÓN DE COLUMNAS---
print("🧹 Seleccionando solo la información ...")

df_bronze = df_raw.select(
    "YEAR", 
    "MONTH", 
    "DAY",
    "DAY_OF_WEEK",
    "AIRLINE", 
    "FLIGHT_NUMBER",
    "TAIL_NUMBER",
    "ORIGIN_AIRPORT", 
    "DESTINATION_AIRPORT",
    "SCHEDULED_DEPARTURE",
    "DEPARTURE_TIME",
    "DEPARTURE_DELAY",
    "ARRIVAL_DELAY",
    "DISTANCE",
    "AIR_TIME",
    "CANCELLED",
    "CANCELLATION_REASON"
)

print("✅ DataFrame 'df_bronze' creado.")
print("--- Muestra de las primeras 5 filas ---")
df_bronze.show(2, vertical=True)

In [4]:
print("--- ✈️ INSPECCIONANDO DIMENSIONES ---")

# Cargar Aerolíneas (Diccionario de compañías)
df_airlines_raw = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("s3a://bronze/raw_data/airlines.csv")

print("\n--- 1. AEROLÍNEAS (Schema & Data) ---")
df_airlines_raw.printSchema()
df_airlines_raw.show(5, truncate=False) 

# Cargar Aeropuertos (Diccionario de lugares + Coordenadas)
df_airports_raw = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("s3a://bronze/raw_data/airports.csv")

print("\n--- 2. AEROPUERTOS (Schema & Data) ---")
df_airports_raw.printSchema()
df_airports_raw.show(5)

--- ✈️ INSPECCIONANDO DIMENSIONES ---

--- 1. AEROLÍNEAS (Schema & Data) ---
root
 |-- IATA_CODE: string (nullable = true)
 |-- AIRLINE: string (nullable = true)

+---------+----------------------+
|IATA_CODE|AIRLINE               |
+---------+----------------------+
|UA       |United Air Lines Inc. |
|AA       |American Airlines Inc.|
|US       |US Airways Inc.       |
|F9       |Frontier Airlines Inc.|
|B6       |JetBlue Airways       |
+---------+----------------------+
only showing top 5 rows


--- 2. AEROPUERTOS (Schema & Data) ---
root
 |-- IATA_CODE: string (nullable = true)
 |-- AIRPORT: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- COUNTRY: string (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)

+---------+--------------------+-----------+-----+-------+--------+----------+
|IATA_CODE|             AIRPORT|       CITY|STATE|COUNTRY|LATITUDE| LONGITUDE|
+---------+---------

In [7]:
# --- CREACIÓN DE DIMENSIONES LIMPIAS ---

# Aerolíneas: Nos quedamos con todo (Código y Nombre)
print("🧹 Limpiando tabla Aerolíneas...")
df_airlines = df_airlines_raw.select(
    df_airlines_raw["IATA_CODE"].alias("AIRLINE_ID"), # Renombramos para evitar confusiones
    df_airlines_raw["AIRLINE"].alias("AIRLINE_NAME")
)

# Aeropuertos: Quitamos COUNTRY
print("🧹 Limpiando tabla Aeropuertos (Eliminando 'Country')...")
df_airports = df_airports_raw.select(
    "IATA_CODE",
    "AIRPORT",
    "CITY",
    "STATE",
    "LATITUDE",
    "LONGITUDE"
)


🧹 Limpiando tabla Aerolíneas...
🧹 Limpiando tabla Aeropuertos (Eliminando 'Country')...
✅ Dimensiones listas.

--- Aerolíneas (Final) ---
+----------+----------------------+
|AIRLINE_ID|AIRLINE_NAME          |
+----------+----------------------+
|UA        |United Air Lines Inc. |
|AA        |American Airlines Inc.|
|US        |US Airways Inc.       |
+----------+----------------------+
only showing top 3 rows

--- Aeropuertos (Final) ---
+---------+--------------------+-----------+-----+--------+----------+
|IATA_CODE|             AIRPORT|       CITY|STATE|LATITUDE| LONGITUDE|
+---------+--------------------+-----------+-----+--------+----------+
|      ABE|Lehigh Valley Int...|  Allentown|   PA|40.65236|  -75.4404|
|      ABI|Abilene Regional ...|    Abilene|   TX|32.41132|  -99.6819|
|      ABQ|Albuquerque Inter...|Albuquerque|   NM|35.04022|-106.60919|
+---------+--------------------+-----------+-----+--------+----------+
only showing top 3 rows



In [9]:
# PRUEBAS CAPA SILVER

# Cargar los DATOS OPTIMIZADOS (Parquet) de la capa Bronze
print("📂 Cargando tablas Bronze (Formatted)...")
df_vuelos = spark.read.parquet("s3a://bronze/formatted/flights")
df_airlines = spark.read.parquet("s3a://bronze/formatted/airlines")
df_airports = spark.read.parquet("s3a://bronze/formatted/airports")
print("✅ Tablas listas en memoria:")


📂 Cargando tablas Bronze (Formatted)...
✅ Tablas listas en memoria:


In [10]:
print("\n📋 Esquema de VUELOS (df_vuelos):")
df_vuelos.printSchema()

print("\n📋 Esquema de AEROLÍNEAS (df_airlines):")
df_airlines.printSchema()

print("\n📋 Esquema de AEROPUERTOS (df_airports):")
df_airports.printSchema()


📋 Esquema de VUELOS (df_vuelos):
root
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: integer (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: integer (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- ARRIVAL_DELAY: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- CANCELLED: integer (nullable = true)
 |-- CANCELLATION_REASON: string (nullable = true)


📋 Esquema de AEROLÍNEAS (df_airlines):
root
 |-- AIRLINE_ID: string (nullable = true)
 |-- AIRLINE_NAME: string (nullable = true)


📋 Esquema de AEROPUERTOS (df_airports):
root
 |-- IATA_CODE: string (nullable =

In [11]:
# Unir flights con airlines: "left" significa: Quédate con todos los vuelos, y si encuentras la aerolínea, pega su nombre.
df_step1 = df_vuelos.join(
    df_airlines, 
    df_vuelos["AIRLINE"] == df_airlines["AIRLINE_ID"], 
    "left"
)

# Limpieza: Ahora tenemos 'AIRLINE' (código) y 'AIRLINE_ID' (código repetido): Borramos la repetida para no ensuciar.
df_step1 = df_step1.drop("AIRLINE_ID")

print("✅ Paso 1 completado. Muestra con nombre de aerolínea:")
df_step1.select("FLIGHT_NUMBER", "AIRLINE", "AIRLINE_NAME").show(5)

✅ Paso 1 completado. Muestra con nombre de aerolínea:
+-------------+-------+--------------------+
|FLIGHT_NUMBER|AIRLINE|        AIRLINE_NAME|
+-------------+-------+--------------------+
|           98|     AS|Alaska Airlines Inc.|
|         2336|     AA|American Airlines...|
|          840|     US|     US Airways Inc.|
|          258|     AA|American Airlines...|
|          135|     AS|Alaska Airlines Inc.|
+-------------+-------+--------------------+
only showing top 5 rows



In [12]:
# Unir flights con airports: el problema es que tenemos que unir dos columnas, por lo que hay variables que se nos duplicarán
from pyspark.sql.functions import col

# ORIGEN
# Usamos .alias("origen") para decirle a Spark: "Trata esta copia de aeropuertos como 'origen'"
print("📍 Cruzando datos del Aeropuerto de ORIGEN...")

df_step2 = df_step1.join(
    df_airports.alias("origen"), 
    col("ORIGIN_AIRPORT") == col("origen.IATA_CODE"),
    "left"
)

# DESTINO
# Usamos .alias("destino") para la segunda vez
print("📍 Cruzando datos del Aeropuerto de DESTINO...")

df_silver = df_step2.join(
    df_airports.alias("destino"), 
    col("DESTINATION_AIRPORT") == col("destino.IATA_CODE"),
    "left"
)

print("✅ Joins completados.")

📍 Cruzando datos del Aeropuerto de ORIGEN...
📍 Cruzando datos del Aeropuerto de DESTINO...
✅ Joins completados.


In [20]:
# CAPA GOLD
print("📂 Cargando Silver Master...")
df_silver = spark.read.parquet("s3a://silver/master_flights")
print("📋 ESTRUCTURA DE LA CAPA SILVER (Master Table):")
df_silver.printSchema()

# KPI 1: HEATMAP DE FIABILIDAD
from pyspark.sql.functions import avg, count, round, col, when

# 3. KPI 1 MEJORADO: Tratando los adelantos como 0
print("🔥 Generando KPI 1 (Corregido: Los adelantos cuentan como 0 retraso)...")

df_heatmap = df_silver.groupBy("DAY_NAME", "DEPARTURE_HOUR") \
    .agg(
        # Si es negativo (< 0), cuenta como 0. Si no, usa el valor real.
        round(
            avg(
                when(col("DEPARTURE_DELAY") < 0, 0)
                .otherwise(col("DEPARTURE_DELAY"))
            ), 2
        ).alias("RETRASO_MEDIO"),
        
        count("*").alias("NUM_VUELOS")
    ) \
    .orderBy("DAY_NAME", "DEPARTURE_HOUR")

print("✅ Tabla Gold KPI 1 (Fiable) lista:")
df_heatmap.show(10, truncate=False)

📂 Cargando Silver Master...
📋 ESTRUCTURA DE LA CAPA SILVER (Master Table):
root
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- DAY_NAME: string (nullable = true)
 |-- AIRLINE_CODE: string (nullable = true)
 |-- AIRLINE_NAME: string (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- FLIGHT_NUMBER: integer (nullable = true)
 |-- ORIGIN_CODE: string (nullable = true)
 |-- ORIGIN_NAME: string (nullable = true)
 |-- ORIGIN_CITY: string (nullable = true)
 |-- ORIGIN_STATE: string (nullable = true)
 |-- ORIGIN_LAT: double (nullable = true)
 |-- ORIGIN_LONG: double (nullable = true)
 |-- DEST_CODE: string (nullable = true)
 |-- DEST_NAME: string (nullable = true)
 |-- DEST_CITY: string (nullable = true)
 |-- DEST_STATE: string (nullable = true)
 |-- DEST_LAT: double (nullable = true)
 |-- DEST_LONG: double (nullable = true)
 |-- SCHEDULED_DEPARTURE: integer (nullable 